In [24]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

import imageio as io
import matplotlib.pyplot as plt
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations
from vis.visualization import visualize_activation,visualize_cam, overlay
import matplotlib.cm as cm

import os


In [0]:
mobile = keras.applications.mobilenet.MobileNet()


In [0]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [28]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x)  #dense layer 3
x=Dense(256,activation='relu')(x)
x=Dense(124,activation='relu')(x)
preds=Dense(49,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [30]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_6
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [31]:
#freeze original mobilenet layers 
for layer in base_model.layers:
  if layer.name == 'conv_pw_13_relu':
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' frozen.')

Capa input_6 frozen.
Capa conv1_pad frozen.
Capa conv1 frozen.
Capa conv1_bn frozen.
Capa conv1_relu frozen.
Capa conv_dw_1 frozen.
Capa conv_dw_1_bn frozen.
Capa conv_dw_1_relu frozen.
Capa conv_pw_1 frozen.
Capa conv_pw_1_bn frozen.
Capa conv_pw_1_relu frozen.
Capa conv_pad_2 frozen.
Capa conv_dw_2 frozen.
Capa conv_dw_2_bn frozen.
Capa conv_dw_2_relu frozen.
Capa conv_pw_2 frozen.
Capa conv_pw_2_bn frozen.
Capa conv_pw_2_relu frozen.
Capa conv_dw_3 frozen.
Capa conv_dw_3_bn frozen.
Capa conv_dw_3_relu frozen.
Capa conv_pw_3 frozen.
Capa conv_pw_3_bn frozen.
Capa conv_pw_3_relu frozen.
Capa conv_pad_4 frozen.
Capa conv_dw_4 frozen.
Capa conv_dw_4_bn frozen.
Capa conv_dw_4_relu frozen.
Capa conv_pw_4 frozen.
Capa conv_pw_4_bn frozen.
Capa conv_pw_4_relu frozen.
Capa conv_dw_5 frozen.
Capa conv_dw_5_bn frozen.
Capa conv_dw_5_relu frozen.
Capa conv_pw_5 frozen.
Capa conv_pw_5_bn frozen.
Capa conv_pw_5_relu frozen.
Capa conv_pad_6 frozen.
Capa conv_dw_6 frozen.
Capa conv_dw_6_bn frozen.


In [32]:
train_datagen=ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True) 

train_generator=train_datagen.flow_from_directory('/content/gdrive/My Drive/Findit/Death Valley/MobileNetData',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=128,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 519 images belonging to 49 classes.


In [0]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=20)

Epoch 1/20
4/4 [==============================] - 82s 20s/step - loss: 3.8500 - acc: 0.0547
Epoch 2/20
4/4 [==============================] - 50s 13s/step - loss: 3.6546 - acc: 0.0892
Epoch 3/20
4/4 [==============================] - 48s 12s/step - loss: 3.4371 - acc: 0.1349
Epoch 4/20
4/4 [==============================] - 48s 12s/step - loss: 3.0973 - acc: 0.2054
Epoch 5/20
4/4 [==============================] - 46s 12s/step - loss: 2.6978 - acc: 0.2344
Epoch 6/20
4/4 [==============================] - 62s 16s/step - loss: 2.3354 - acc: 0.3184
Epoch 7/20
4/4 [==============================] - 50s 12s/step - loss: 1.7974 - acc: 0.4523
Epoch 8/20
4/4 [==============================] - 48s 12s/step - loss: 1.8581 - acc: 0.4793
Epoch 9/20
4/4 [==============================] - 47s 12s/step - loss: 1.5152 - acc: 0.4647
Epoch 10/20
4/4 [==============================] - 47s 12s/step - loss: 1.6434 - acc: 0.5186
Epoch 11/20
4/4 [==============================] - 62s 16s/step - loss: 1.5759 

In [0]:
gpath='/content/gdrive/My Drive/Findit/Death Valley/'
#save model in disk
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open(gpath+'modelv7.json', "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(gpath+'modelv7.h5')
print("Saved model to disk")